# Time Series Data Analysis For Air Quality using Auto Regressive Model

# Air Quality in Dar es Salaam

In [3]:
# Import libraries here
import warnings 
warnings.simplefilter(action="ignore", category=FutureWarning) 
import inspect
import time
import warnings

import matplotlib.pyplot as plt
import pandas as pd
# import plotly.express as px
import seaborn as sns
from IPython.display import VimeoVideo
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa.arima.model import ARIMA

warnings.filterwarnings("ignore")

# Prepare Data
### Connect

#### Task 1: 
Connect to MongoDB server running at host "localhost" on port 27017. Then connect to the "air-quality" database and assign the collection for Dar es Salaam to the variable name dar

In [13]:
client = MongoClient(host="localhost",port=27017)
db = client["air-quality"]
dar = db["dar-es-salaam"]

### Task 2: 
Complete the wrangle function below so that the results from the database query are read into the DataFrame df. Be sure that the index of df is the "timestamp" from the results.

In [9]:
def wrangle(collection):
    results = collection.find(
        {"metadata.site": 29, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )
    df = pd.DataFrame(results).set_index("timestamp")
    
    df.index=df.index.tz_localize("UTC").tz_convert("Africa/Nairobi")
    
    df=df[df["P2"]<500]
    
    df=df["P2"].resample("1H").mean().fillna(method="ffill").to_frame()
    
    df["P2.L1"]=df["P2"].shift(1)
    df.dropna(inplace=True)

    return df

### Task 3:
Use your wrangle function to read the data from the nairobi collection into the DataFrame df.

In [24]:
# df = wrangle(dar)
# df.head(10)

### Explore

##### Task 4:
Determine the numbers assigned to all the sensor sites in the Dar es Salaam collection. Your submission should be a list of integers.

In [25]:
sites = dar.distinct("metadata.site")
sites

#### Task 5: 
Determine which site in the Dar es Salaam collection has the most sensor readings (of any type, not just PM2.5 readings). You submission readings_per_site should be a list of dictionaries that follows this format:

[{'_id': 6, 'count': 70360}, {'_id': 29, 'count': 131852}]

Note that the values here ☝️ are from the Nairobi collection, so your values will look different.

In [ ]:
result = dar.aggregate(
    [
     {"$group":{"_id":"$metadata.site","count":{"$count":{}}}}   
    ]
)
readings_per_site = list(result)
readings_per_site

# Import

### Task 6:
Create a wrangle function that will extract the PM2.5 readings from the site that has the most total readings in the Dar es Salaam collection. Your function should do the following steps:
<ol>
<li>Localize reading time stamps to the timezone for "Africa/Dar_es_Salaam".</li>
<li>Remove all outlier PM2.5 readings that are above 100.</li>
<li>Resample the data to provide the mean PM2.5 reading for each hour.</li>
<li>Impute any missing values using the forward-will method.</li>
<li>Return a Series y.</li>
</ol>

In [19]:
def wrangle(collection,resample_rule="1H"): 
    results = collection.find(
        {"metadata.site": 11, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )
        # Read results into DataFrame
    df = pd.DataFrame(list(results)).set_index("timestamp")
    # Localize timezone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")
    # Remove outliers
    df = df[df["P2"] < 100]
    # Resample and forward-fill
    y = df["P2"].resample(resample_rule).mean().fillna(method="ffill")
    # Read results into DataFrame
#     df = pd.DataFrame(list(results)).set_index("timestamp")
    return y

Use your wrangle function to query the dar collection and return your cleaned results.

In [ ]:
y = wrangle(dar)
y.head()

# Explore Some More

### Task 7: 
Create a time series plot of the readings in y. Label your x-axis "Date" and your y-axis "PM2.5 Level". Use the title "Dar es Salaam PM2.5 Levels".

In [22]:
fig, ax = plt.subplots(figsize=(15, 6))
y.plot(xlabel="Date",ylabel="PM2.5 Level",title="Dar es Salaam PM2.5 Levels",ax=ax)
# Don't delete the code below 👇
plt.savefig("images/3-5-5.png", dpi=150)

### Task 8:
Plot the rolling average of the readings in y. Use a window size of 168 (the number of hours in a week). Label your x-axis "Date" and your y-axis "PM2.5 Level". Use the title "Dar es Salaam PM2.5 Levels, 7-Day Rolling Average".

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.rolling(168).mean().plot(ax=ax)
# Don't delete the code below 👇
plt.savefig("images/3-5-6.png", dpi=150)

### Task 8: 
Create an ACF plot for the data in y. Be sure to label the x-axis as "Lag [hours]" and the y-axis as "Correlation Coefficient". Use the title "Dar es Salaam PM2.5 Readings, ACF".

In [ ]:

fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y,ax=ax)
# Don't delete the code below 👇
plt.savefig("images/3-5-7.png", dpi=150)

### Task 9: 
Create an PACF plot for the data in y. Be sure to label the x-axis as "Lag [hours]" and the y-axis as "Correlation Coefficient". Use the title "Dar es Salaam PM2.5 Readings, PACF".

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y,ax=ax)
# Don't delete the code below 👇
plt.savefig("images/3-5-8.png", dpi=150)

## Split

#### Task 10:
Split y into training and test sets. The first 90% of the data should be in your training set. The remaining 10% should be in the test set.

In [ ]:
cutoff_test = int(len(y)*0.9)
y_train =  y.iloc[:cutoff_test]

print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
y_test = y.iloc[cutoff_test:]

# Build Model

### Baseline

### Task 11:
Establish the baseline mean absolute error for your model.

In [ ]:
y_train_mean =y_train.mean()
y_pred_baseline =  [y_train_mean]*len(y_train)
mae_baseline = mean_absolute_error(y_train,y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

### Iterate

### Task 12: 
You're going to use an AR model to predict PM2.5 readings, but which hyperparameter settings will give you the best performance? Use a for loop to train your AR model on using settings for p from 1 to 30. Each time you train a new model, calculate its mean absolute error and append the result to the list maes. Then store your results in the Series mae_series.

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
p_params = range(1,3) 
maes = []
for p in p_params: 
#     maes[p]= list() 
        # Note start time
        # Train model
    model = AutoReg(y_train,lags=26,old_names=False).fit()
#         model = ARIMA(y_train).fit()
    y_pred = model.predict().dropna()
#     print(y_pred[1])
            # Calculate training MAE
#     mean_absolute_error(y_train.iloc[26:],y_pred)
#     x=mean_absolute_error(y_train.iloc[26:],y_pred)
    x=mean_absolute_error(y_train, y_pred_baseline)
    maes.append(x)
# maes.append(mean_absolute_error(y_train,y_pred_baseline))
mae_series = pd.Series(maes, name="mae", index=p_params)
mae_series

### Task 13: 
Look through the results in mae_series and determine what value for p provides the best performance. Then build and train final_model using the best hyperparameter value.

Note: Make sure that you build and train your model in one line of code, and that the data type of best_model is statsmodels.tsa.ar_model.AutoRegResultsWrapper.

In [ ]:
best_p = ...
best_model = ...

### Task 14: 
Calculate the training residuals for best_model and assign the result to y_train_resid. Note that your name of your Series should be "residuals".

In [ ]:
y_train_resid = y_train-y_pred
# y_train_resid.tail()
# y_train_resid = ...
y_train_resid.name = "residuals"
y_train_resid.head()
y_train_resid = y_train-y_pred
# y_train_resid.tail()
# y_train_resid = ...
y_train_resid.name = "residuals"
y_train_resid.head()

### Task 15: 
Create a histogram of y_train_resid. Be sure to label the x-axis as "Residuals" and the y-axis as "Frequency". Use the title "Best Model, Training Residuals".

In [ ]:
Plot histogram of residuals
y_train_resid.plot(ax=ax)
# Don't delete the code below 👇
plt.savefig("images/3-5-14.png", dpi=150)

### Task 16: 
Create an ACF plot for y_train_resid. Be sure to label the x-axis as "Lag [hours]" and y-axis as "Correlation Coefficient". Use the title "Dar es Salaam, Training Residuals ACF".

In [ ]:

fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y_train_resid,ax=ax);
# Don't delete the code below 👇
plt.savefig("images/3-5-15.png", dpi=150)

# Evaluate 

### Task 17:
Perform walk-forward validation for your model for the entire test set y_test. Store your model's predictions in the Series y_pred_wfv. Make sure the name of your Series is "prediction" and the name of your Series index is "timestamp".

In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    model=AutoReg(history,lags=26).fit()
    next_pred=model.forecast()
    y_pred_wfv=y_pred_wfv.append(next_pred)
    history=history.append(y_test[next_pred.index])
#     pass
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

### Task 18:
Submit your walk-forward validation predictions to the grader to see test mean absolute error for your model.

### Communicate Results

### Task 19: 
Put the values for y_test and y_pred_wfv into the DataFrame df_pred_test (don't forget the index). Then plot df_pred_test using plotly express. Be sure to label the x-axis as "Date" and the y-axis as "PM2.5 Level". Use the title "Dar es Salaam, WFV Predictions".

In [ ]:
df_pred_test = pd.DataFrame({"y_test":y_test,"y_pred_wfv":y_pred_wfv})
fig = fig = px.line(df_pred_test,labels={"value":"PM2.5"})
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)
# Don't delete the code below 👇
fig.write_image("images/3-5-18.png", scale=1, height=500, width=700)
​
fig.show()